In [1]:
from sklearn.ensemble import RandomForestClassifier
from joblib import load
from sklearn.model_selection import KFold, cross_val_score

In [4]:
combined_data = load('..../DataDumps/SpectralSubt.joblib')
combined_labels = load('..../DataDumps/labels.joblib')

FileNotFoundError: [Errno 2] No such file or directory: '..../DataDumps/SpectralSubt.joblib'

In [3]:
def reshape_data(data):
    num_samples = data.shape[0]
    num_timesteps = data.shape[1]
    num_channels = data.shape[2]  # Assuming the 3rd dimension is the number of channels
    return data.reshape(num_samples, num_timesteps * num_channels) 


combined_data1 = reshape_data(combined_data)  

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_data1, combined_labels, test_size=0.4) 

In [5]:
rf = RandomForestClassifier(n_estimators=100)

In [6]:
k_folds = KFold(n_splits = 5)

scores = cross_val_score(rf, combined_data1, combined_labels, cv = k_folds)

In [7]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.17708333 0.28125    0.51041667 0.93229167 0.86458333]
Average CV Score:  0.553125
Number of CV Scores used in Average:  5


In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, predictions1)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, predictions1)
print("Confusion Matrix:\n", cm)

print(classification_report(y_test, predictions1))


NameError: name 'predictions1' is not defined

# Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier

# Define a parameter distribution
param_distributions = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

# Create a RandomizedSearchCV object
rf_random = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_distributions
)

# Fit on the training data 
rf_random.fit(X_train, y_train)

# Get the best model
best_rf = rf_random.best_estimator_


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# Define the parameter grid
param_dist = {
    'n_estimators': randint(100, 1000),  # Number of trees in the forest
    'max_depth': randint(1, 50),         # Maximum depth of the tree
    'min_samples_split': randint(2, 20), # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 20),  # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2', None],  # Number of features to consider when looking for the best split
    'bootstrap': [True, False]           # Whether bootstrap samples are used when building trees
}

# Initialize Random Forest classifier
rf = RandomForestClassifier()

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1
)

# Perform random search
random_search.fit(X_train, y_train)  # Assuming X_train and y_train are your training data

# Print the best parameters found
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_rf = random_search.best_estimator_

# Evaluate the best model
accuracy = best_rf.score(X_test, y_test)  # Assuming X_test and y_test are your test data
print("Accuracy on test set:", accuracy)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
rf_random.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_split': 2,
 'n_estimators': 50}

In [ ]:
predictions2 = best_rf.predict(X_test)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, predictions2)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, predictions2)
print("Confusion Matrix:\n", cm)

print(classification_report(y_test, predictions2))


NameError: name 'X_test' is not defined

3 class

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from joblib import load
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load data
combined_data = load('../DataDumps/data_fft.joblib')
combined_labels = load('../DataDumps/labels.joblib')

# Define a function to reshape the data
def reshape_data(data):
    num_samples = data.shape[0]
    num_timesteps = data.shape[1]
    num_channels = data.shape[2]
    return data.reshape(num_samples, num_timesteps * num_channels) 

combined_data1 = reshape_data(combined_data)  

# Split data into training/validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(combined_data1, combined_labels, test_size=0.2, stratify=combined_labels, random_state=0)

# Define K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [1, 2, 4],
    #'max_features': ['auto', 'sqrt', 'log2']
} 

# Initialize Random Forest classifier
rf = RandomForestClassifier()

# Initialize variables to store best hyperparameters and accuracies
best_hyperparams = []
best_accuracies = []

# Perform hyperparameter tuning and K-Fold cross-validation
for train_index, val_index in kf.split(X_train_val):
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    y_train, y_val = y_train_val[train_index], y_train_val[val_index]
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(rf, param_grid, cv=kf, n_jobs=-1)
    
    # Perform GridSearchCV
    grid_search.fit(X_train, y_train)
    
    # Get the best Random Forest model from GridSearchCV
    best_rf = grid_search.best_estimator_
    
    # Store best hyperparameters
    best_hyperparams.append(grid_search.best_params_)
    
    # Make predictions on validation set
    predictions_val = best_rf.predict(X_val)
    
    # Calculate accuracy on validation set
    accuracy_val = accuracy_score(y_val, predictions_val)
    
    # Print confusion matrix and classification report for each fold
    cm = confusion_matrix(y_val, predictions_val)
    print("Confusion Matrix for validation set:\n", cm)
    print(classification_report(y_val, predictions_val))
    
    # Make predictions on test set
    predictions_test = best_rf.predict(X_test)
    
    # Calculate accuracy on test set
    accuracy_test = accuracy_score(y_test, predictions_test)
    best_accuracies.append(accuracy_test)
    
    # Print confusion matrix and classification report for test set
    cm_test = confusion_matrix(y_test, predictions_test)
    print("Confusion Matrix for test set:\n", cm_test)
    print(classification_report(y_test, predictions_test))

# Print average accuracy across all folds for test set
print("Average Accuracy on Test Set:", np.mean(best_accuracies))

# Print the best hyperparameters found during each fold
print("Best hyperparameters for each fold:")
for i, params in enumerate(best_hyperparams):
    print("Fold {}: {}".format(i+1, params))


Confusion Matrix for validation set:
 [[72  0  1]
 [ 5 28  2]
 [23 15  8]]
              precision    recall  f1-score   support

           0       0.72      0.99      0.83        73
           1       0.65      0.80      0.72        35
           2       0.73      0.17      0.28        46

    accuracy                           0.70       154
   macro avg       0.70      0.65      0.61       154
weighted avg       0.71      0.70      0.64       154

Confusion Matrix for test set:
 [[89  6  1]
 [ 5 42  2]
 [25 12 10]]
              precision    recall  f1-score   support

           0       0.75      0.93      0.83        96
           1       0.70      0.86      0.77        49
           2       0.77      0.21      0.33        47

    accuracy                           0.73       192
   macro avg       0.74      0.67      0.64       192
weighted avg       0.74      0.73      0.69       192

Confusion Matrix for validation set:
 [[77  1  3]
 [ 4 32  1]
 [23 11  2]]
              preci

2 class

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from joblib import load
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load data
combined_data = load('../DataDumps/data_fft.joblib')
combined_labels = load('../DataDumps/labels.joblib')
combined_labels[combined_labels == 2] = 1

# Define a function to reshape the data
def reshape_data(data):
    num_samples = data.shape[0]
    num_timesteps = data.shape[1]
    num_channels = data.shape[2]
    return data.reshape(num_samples, num_timesteps * num_channels) 

combined_data1 = reshape_data(combined_data)  

# Split data into training/validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(combined_data1, combined_labels, test_size=0.2, stratify=combined_labels, random_state=0)

# Define K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [1, 2, 4],
    #'max_features': ['auto', 'sqrt', 'log2']
} 

# Initialize Random Forest classifier
rf = RandomForestClassifier()

# Initialize variables to store best hyperparameters and accuracies
best_hyperparams = []
best_accuracies = []

# Perform hyperparameter tuning and K-Fold cross-validation
for train_index, val_index in kf.split(X_train_val):
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    y_train, y_val = y_train_val[train_index], y_train_val[val_index]
    
    # Initialize GridSearchCV
    grid_search = GridSearchCV(rf, param_grid, cv=kf, n_jobs=-1)
    
    # Perform GridSearchCV
    grid_search.fit(X_train, y_train)
    
    # Get the best Random Forest model from GridSearchCV
    best_rf = grid_search.best_estimator_
    
    # Store best hyperparameters
    best_hyperparams.append(grid_search.best_params_)
    
    # Make predictions on validation set
    predictions_val = best_rf.predict(X_val)
    
    # Calculate accuracy on validation set
    accuracy_val = accuracy_score(y_val, predictions_val)
    
    # Print confusion matrix and classification report for each fold
    cm = confusion_matrix(y_val, predictions_val)
    print("Confusion Matrix for validation set:\n", cm)
    print(classification_report(y_val, predictions_val))
    
    # Make predictions on test set
    predictions_test = best_rf.predict(X_test)
    
    # Calculate accuracy on test set
    accuracy_test = accuracy_score(y_test, predictions_test)
    best_accuracies.append(accuracy_test)
    
    # Print confusion matrix and classification report for test set
    cm_test = confusion_matrix(y_test, predictions_test)
    print("Confusion Matrix for test set:\n", cm_test)
    print(classification_report(y_test, predictions_test))

# Print average accuracy across all folds for test set
print("Average Accuracy on Test Set:", np.mean(best_accuracies))

# Print the best hyperparameters found during each fold
print("Best hyperparameters for each fold:")
for i, params in enumerate(best_hyperparams):
    print("Fold {}: {}".format(i+1, params))


Confusion Matrix for validation set:
 [[68  5]
 [25 56]]
              precision    recall  f1-score   support

           0       0.73      0.93      0.82        73
           1       0.92      0.69      0.79        81

    accuracy                           0.81       154
   macro avg       0.82      0.81      0.80       154
weighted avg       0.83      0.81      0.80       154

Confusion Matrix for test set:
 [[82 14]
 [20 76]]
              precision    recall  f1-score   support

           0       0.80      0.85      0.83        96
           1       0.84      0.79      0.82        96

    accuracy                           0.82       192
   macro avg       0.82      0.82      0.82       192
weighted avg       0.82      0.82      0.82       192

Confusion Matrix for validation set:
 [[75  6]
 [18 55]]
              precision    recall  f1-score   support

           0       0.81      0.93      0.86        81
           1       0.90      0.75      0.82        73

    accuracy     